In [ ]:
import numpy as np

## Helper Functions

### Convert String to Int Array (ASCII)

In [ ]:
def stringToIntArr(str):
    byte_st = str.encode('ascii')
    return np.array(list(map(int, byte_st)))

### Convert Int Array (ASCII) to String

In [ ]:
def intArrToString(intArr):
    return (''.join(chr(i) for i in intArr))

### Convert Int Array to Single Int

In [ ]:
def byteArrToSingleInt(byte_arr_in):
    '''
    ASCII codes can be up to 3 digits long (in decimal).
    To enable effective decryption, ASCII codes with less than 3 digits
    will be padded at the front with extra 0's.
    Then all codes will be concatenated into one long string of digits,
    and this will be cast into an integer.
    
    Input:
        byte_arr_int - array of integers
    
    Output:
        x_int - a single integer containing all integers in the input array
    '''
    x_st = ""
    for i in byte_arr_in:
        i_st = str(i)
        if len(i_st) == 1:
            i_st = "00" + i_st
        elif len(i_st) == 2:
            i_st = "0" + i_st
        x_st = x_st + i_st
        x_int = int(x_st)
    return x_int

### Convert Single Int to Array of Ints

In [ ]:
def singleIntToByteArr(singleInt):
    int_str = str(singleInt)
    if len(int_str)%3 == 1:
        int_str = "00" + int_str
    elif len(int_str)%3 == 2:
        int_str = "0" + int_str
    chars = len(int_str)//3
    int_arr = []

    for i in range(chars):
        # remove first three characters, put in its own place in the array
        int_arr.append(int(int_str[3*i:3*(i+1)]))
    return int_arr

### Modular Exponentiation

In [ ]:
def modExp(g,x,p):
    '''
    An efficient way to raise a number to a large power, modulus some 
    other integer: g^x mod p
    
    Complexity is O(log(x)), logarithmic complexity, compared to computing
    g^x mod p directly, which is O(x), linear complexity.
    
    Input:
        g - integer base, to be multiplied by itself x times
        x - integer exponent
        p - modulus, or divisor
    
    Output:
        r - equal to g^x mod p
    '''
    
    c = g % p
    d = x
    r = 1

    while d > 0:
        if d % 2 == 1:
            r = (r*c) % p
        d = d//2
        c = (c*c) % p
    return r

### Euclidean Algorithm

In [ ]:
def euclidean(a,b):
    '''
    Finds the Greatest Common Divisor of two integers, (a, b)
    
    Input:
        a, b - both integers
        
    Output:
        GCD(a,b)
    '''
    if b == 0:
        return a
    else:
        return euclidean(b, a % b)

### Extended Euclidean Algorithm

In [ ]:
def extendedEuclidean(a,b):
    '''
    Finds the Greatest Common Divisor of two integers, (a, b), and finds
    integers x and y such that:
    
    x*a + y*b = GCD(a,b)
    
    Thereby expressing the GCD(a,b) as a linear combination of a and b.
    
    Input:
        a, b - both integers
        
    Output:
        r - GCD(a,b)
        x - factor to multiply a by
        y - factor to multiply y by
    '''
    if b == 0:
        return (a,1,0)
    else:
        (r,x,y) = extendedEuclidean(b, a % b)
        return (r, y, x-y*(a//b))

## RSA Implementation

### RSA Encryption

In [ ]:
def encryptRSA(M,n,e):
    '''
    Encrypts a plaintext message expressed as an integer according to 
    RSA protocol: C = M^e mod n
    
    Input:
        M - plaintext message, which must be smaller than modulus
        n - modulus for public & private key, product of two large primes
        e - exponent for public key
        
    Output:
        C - ciphertext message
    '''
    assert M < n
    C = modExp(M,e,n)
    return C

### RSA Decryption

In [ ]:
def decryptRSA(C,n,d):
    '''
    Decrypts a ciphertext message expressed as an integer according to 
    RSA protocol: M = C^d mod n
    
    Input:
        C - ciphertext message
        n - modulus for public & private key, product of two large primes
        d - part of private key
        
    Output:
        M - plaintext message
    '''
    
    M = modExp(C,d,n)
    return M

## RSA Examples

### RSA Example with small primes
RSA require two prime numbers to start. The modulus used for encryption and decryption is the product of those primes.
> $n=p*q$

We will also calculate the "totient" of n, $\phi(n)$, to help us set up the rest of the scheme.
> $\phi(n)=(p-1)*(q-1)$

Note that knowledge of $p$ and $q$ is required to find $n$ and $\phi(n)$.

In [ ]:
p, q = 17,23
n = p * q
totient = (p-1)*(q-1)
print("Large product of primes, p*q=n, is",n)
print("Totient value, (p-1)*(q-1), is",totient)

Now that we have the modulus, we need exponents to complete the public and private key pairs.

**Public key:**
>$GCD(e,\phi(n)) = 1$

i.e. no shared factors between e and $\phi(n)$. Since $p$ and $q$ are always odd, $\phi(n)$ will always be even, so an odd prime is a good choice to ensure there are no shared factors (e.g. 3).

**Private key**
> $e*d$ $mod$ $\phi(n) = 1$

i.e. $e*d + y*\phi(n) = 1$ (where $y$ is some integer). This is like writing the $GCD(e,\phi(n))$ as a linear combination of $e$ and $\phi(n)$, and $d$ will be the factor multiplied by $e$. Therefore, we can solve for $d$ with the Extended Euclidean algorithm.

In [ ]:
e = 3

(gcd,x,y) = extendedEuclidean(e,totient)
if x < 0:
    d = x + totient
else:
    d = x
print("d =",d)

Now we're ready to encrypt any message that can be expressed as an integer smaller than our $n$.

In [ ]:
small_mess = 314

In [ ]:
C = encryptRSA(small_mess,n,e)
print("Cipertext is",C)

In [ ]:
original_mess = decryptRSA(C,n,d)
print("Cipertext decrypted to reveal original message,",original_mess)

### RSA Example with larger primes
In practice, we want quite large primes to prevent attacks on RSA, about 1000 decimal digits or 4096 bits. Let's see an example with primes that would be difficult to work with by hand.

In [ ]:
p_large = 10152463 # 10,152,463
q_large = 10232143 # 10,232,143
n_large = p_large * q_large
totient_large = (p_large-1) * (q_large-1)
print("Large product of primes, p*q=n, is","{:,}".format(n_large))
print("Totient value, (p-1)*(q-1), is","{:,}".format(totient_large))

In [ ]:
# a popular choice for e is 65,537 = 2^16 + 1, because it is somewhat large, it can be computed quickly since it is
# +1 more than an even power of 2, and it is prime
e_large = 65537

# GCD of e and totient should be 1
euclidean(totient_large,e_large)

In [ ]:
(gcd_large,x_large,y_large) = extendedEuclidean(e_large,totient_large)

if x_large < 0:
    d_large = x_large + totient_large
else:
    d_large = x_large
print("Exponent for private key is","{:,}".format(d_large))

Let's encrypt that small numerical message again, <code>small_mess=314</code>.

In [ ]:
C_1 = encryptRSA(small_mess,n_large,e_large)
print("Ciphertext is",C_1,"or","{:,}".format(C_1))

In [ ]:
M_1 = decryptRSA(C_1,n_large,d_large)
print("Original message, visible after decryption, is",M_1)

### RSA Example with larger primes - encrypting text message
Now let's encrypt a larger message, a string encoded as a large integer.

In [ ]:
textM = "Emily"
intarrM = stringToIntArr(textM)
intM = byteArrToSingleInt(intarrM)
print("Plaintext in original form:",textM)
print("Plaintext as an array of ASCII codes:",intarrM)
print("Plaintext as one large integer:",intM)

In [ ]:
C_2 = encryptRSA(intM,n_large,e_large)
print("Ciphertext is",C_2,"or","{:,}".format(C_2))

In [ ]:
M_2 = decryptRSA(C_2,n_large,d_large)
print("Plaintext as integer, returned by decryption is",M_2)

In [ ]:
intarrM = singleIntToByteArr(M_2)
textM = intArrToString(intarrM)
print("Plaintext returned to array of ASCII chars is",intarrM)
print("Plaintext returned to original readable form is",textM)

## RSA - encrypting vs. signing
Although RSA has some clear limitations (requiring huge prime numbers, performing rather slowly on large messages), it has an additional (and perhaps primary) purpose aside from encrypting information. This same scheme can also be used to digitally "sign" messages, thereby increasing confidence in who sent the message.\
RSA always functions based off a pair of keys, *public* and *private*.\
**Public key** = $(e,n)$ \
**Private key** = $(d,n)$

### Encryption
As seen above, encryption requires the sender to know the public key of their desired recipient. Suppose Alice wants to send Bob a message, and she wants to encrypt it with RSA.
>Alice writes message &rarr; **encrypt** &rarr; transmit to Bob &rarr; **decrypt** &rarr; Bob reads message

Alice needs to know Bob's public key, $(e_{b}, n_{b})$. She will perform encryption in the following manner:
> **Encrypt**: $C = M^{e_{b}}$ $mod$  $n_{b}$

Since Bob is the only person who knows his private key, $(d_{b}, n_{b})$, only he can decrypt the message from Alice. He will perform decryption in the following manner:
> **Decrypt**: $M = C^{d_{b}}$ $mod$  $n_{b}$

### Signatures
Now suppose Alice wants to send Bob a message, and she wants him to trust it was her that sent it. 
> Alice writes message &rarr; **sign** &rarr; transmit to Bob &rarr; **verify** &rarr; Bob reads and trusts message

She needs to use a piece of information *only she* could know, her private key, $(d_{a}, n_{a})$, to sign the message.
> **Sign**: $C = M^{d_{a}}$ $mod$  $n_{a}$

When Bob receives her message, he can use Alice's public key, $(e_{a}, n_{a})$, to verify that the message indeed came from her.
> **Verify**: $M = C^{e_{a}}$ $mod$  $n_{a}$

These two functions may be used in tandem, to ensure both confidentiality of the message and integrity of the sender.